In [1]:
def read_file(path):
    data = []
    with open(path) as f:
        for line in f:
            data.append(line.rstrip())
            
    header = data[0]
    # check delimiter
    delims = [',',';','|','/t']
    
    delim = [x for x in delims if x in header]

    if len(delim) > 1:
        print('Mixed delimiter')
    else:
        delim = delim[0]
        print('Delimiter: {}'.format(delim))
    print('\n')
    return [x.split(delim) for x in data]

In [2]:
def process_wind(data_list):
    print('Processing Wind')
    # wind should be 1) in mph, 2) in last column, 3) have numeric values
    header = [x.upper() for x in data_list[0]]
    wind_label = [x for x in header if 'WIND' in x][0]
    
    # check index
    index = header.index(wind_label)
    if index != 5:
        print('Wind is in column index {}'.format(index))
    
    # check speed
    if 'MPH' in wind_label:
        speed = 'MPH'
    else:
        print('Speed not in MPH')
        speed = wind_label[-3:]
    
    # check values
    # first, take subset
    sub = [x[index] for x in data_list]
    # by visual inspection, some have 'Calm' as a value. replace this value with numeric 0
    cleaned = [float(x) if x.upper() != 'CALM' else 0 for x in sub[1:]]
    
    # also, if negative value, replace it with zero
    cleaned = [0 if x < 0 else x for x in cleaned]
    
    # convert to MPH if KPH
    if speed=='KPH':
        conv = 0.621371
        cleaned = [x*conv for x in cleaned]

    # calculate avg speed for march 2006
    # find a boolean vector where month is March and Year is 2006
    month = [x[header.index('MONTH')] for x in data_list]
    march = [x=='3' for x in month]
    year = [x[header.index('YEAR')] for x in data_list]
    yr_2006 = [x=='2006' for x in year]
    march_2006 = march and yr_2006
    # subset the cleaned wind data using the boolean list
    march_2006_data =  [i for (i, v) in zip(cleaned, march_2006) if v]
    
    minval = min(march_2006_data)
    maxval = max(march_2006_data)
    print('Min: {}, Max: {}'.format(minval, maxval))
    # calc average wind space for this subset
    sigma = 0
    for x in march_2006_data:
        sigma += x
    
    n = len(march_2006_data)
    
    # return the average wind speed
    print('\n')
    return sigma/n

In [3]:
def process_temp(data_list):
    print('Processing Temp')
    # wind should be 1) in mph, 2) in last column, 3) have numeric values
    header = [x.upper() for x in data_list[0]]
    temp_label = [x for x in header if 'TEMPERATURE' in x][0]
    
    # check index
    index = header.index(temp_label)
    if index != 4:
        print('Temp is in column index {}'.format(index))
    
    # check units
    if 'F' in temp_label:
        temp = 'F'
    else:
        print('Temp not in F')
        temp = temp_label[-1:]
    
    # check values
    # first, take subset
    sub = [x[index] for x in data_list]
        
    # ensure numerics
    cleaned = [float(x) for x in sub[1:]]
    
    # convert to C if F temp
    if temp=='C':
        cleaned = [(x*1.8)+32 for x in cleaned]
    
    # calculate avg temp 2016
    # find a boolean vector where year is 2006
    year = [x[header.index('YEAR')] for x in data_list]
    yr_2006 = [x=='2006' for x in year]
    # subset the cleaned temp data using the boolean list
    yr_2006_data =  [i for (i, v) in zip(cleaned, yr_2006) if v]
    
    # check outliers
    print(min(yr_2006_data))
    
    # remove outliers
    yr_2006_data = [x for x in yr_2006_data if x > -150]
    
    minval = min(yr_2006_data)
    maxval = max(yr_2006_data)
    
    print('Min: {}, Max: {}'.format(minval, maxval))
    # calc average wind space for this subset
    sigma = 0
    for x in yr_2006_data:
        sigma += x
    
    n = len(yr_2006_data)
    print('\n')
    # return the average wind speed
    return sigma/n

# Read in all the data. What is wrong with each file's delimiters?
- city "xyz" uses a semicolon delimter

In [4]:
abc = '../Data/ABCWeather.csv'
klm = '../Data/KLMWeather.csv'
pqr = '../Data/PQRWeather.csv'
xyz = '../Data/XYZWeather.txt'
cities = ['abc', 'klm', 'pqr', 'xyz']
all_paths = [abc, klm, pqr, xyz]
all_data = [read_file(x) for x in all_paths]

Delimiter: ,


Delimiter: ,


Delimiter: ,


Delimiter: ;




# Which city has average wind speed closest to 8.30 mph for March, 2006?

- city "xyz" is closest to 8.3 mph

In [6]:
import pandas as pd

In [9]:
p = pd.read_csv(pqr)
p.head()

,Year,Month,Wind SpeedKPH,Day,TimeCST,TemperatureF
0,2000,1,21.45,1,12:53 AM,36.0
1,2000,1,16.25,1,1:53 AM,37.0
2,2000,1,13.05,1,2:53 AM,36.0
3,2000,1,10.39,1,3:53 AM,34.0
4,2000,1,6.23,1,4:53 AM,28.9


In [17]:
p = pd.read_csv(pqr)
p.query('Year==2006 and Month == 3')['Wind SpeedKPH'].mean()*0.621371

8.302345054666674

In [20]:
xyz = pd.read_csv('../Data/XYZWeather.txt',sep=';')
xyz.query('Year==2006 and Month == 3')['Wind SpeedMPH'].mean()

TypeError: Could not convert 3.55.8CalmCalmCalm4.65.86.98.19.211.511.512.711.510.410.49.212.75.85.86.98.18.18.16.98.18.19.26.98.19.28.15.89.210.46.98.15.88.19.29.28.19.210.411.511.513.810.410.413.812.713.811.512.711.511.511.511.58.13.55.85.83.54.68.18.18.16.98.19.212.710.46.94.64.63.54.66.94.65.86.98.18.18.19.26.910.410.48.16.93.5CalmCalmCalmCalm3.53.5Calm5.84.6CalmCalmCalm5.83.56.95.85.85.83.5CalmCalm5.84.63.53.5CalmCalmCalmCalmCalmCalmCalmCalm4.63.5CalmCalmCalm3.54.64.66.94.65.85.88.14.65.84.6Calm5.83.5Calm3.53.53.53.53.53.53.53.55.84.64.64.63.5Calm4.63.54.63.53.53.53.53.5CalmCalm3.5CalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalm3.5CalmCalm6.96.98.18.110.412.712.711.511.512.711.510.410.46.99.29.28.16.96.94.6Calm3.55.84.6CalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalm3.5CalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalmCalm4.610.411.512.79.29.26.98.18.111.51513.810.49.210.412.711.58.19.25.83.5CalmCalmCalmCalmCalmCalmCalmCalm3.54.65.85.84.64.6Cal

In [5]:
wind_results = [process_wind(x) for x in all_data]

Processing Wind
Min: 0, Max: 34.5


Processing Wind
Wind is in column index 4
Min: 0, Max: 33.7


Processing Wind
Wind is in column index 2
Speed not in MPH
Min: 0.0, Max: 28.31587647


Processing Wind
Min: 0, Max: 31.1




In [6]:
wind_diffs = [x-8.3 for x in wind_results]
{k: v for k, v in zip(cities, wind_diffs)}

{'abc': -0.7531180935908148,
 'klm': -0.7503939656000425,
 'pqr': -0.36500456067455644,
 'xyz': 0.06329418559879585}

# Which city had the closest temperature in fahrenheit to 49.65 for all 2006?
- city 2 "KLM" is closest to 49.65

In [7]:
temp_results = [process_temp(x) for x in all_data]

Processing Temp
Temp not in F
-9999.003999999999
Min: -7.996000000000002, Max: 98.096


Processing Temp
Temp is in column index 0
-5223.49
Min: 6.7, Max: 84.87


Processing Temp
Temp is in column index 5
-9999.0
Min: -8.0, Max: 98.1


Processing Temp
-9999.0
Min: -17.0, Max: 96.8




In [8]:
temp_diffs=[x-49.65 for x in temp_results]
{k: v for k, v in zip(cities, temp_diffs)}

{'abc': 2.602219415790927,
 'klm': -0.0016455818446488024,
 'pqr': 2.602193193868686,
 'xyz': -1.585706715482928}